# Ensemble Preperation
This notebook handles the preperation of data for ensemble analysis and exports this prepared data into a shapefile.

In [1]:
import geopandas as gpd
import pandas as pd
import shapely
import numpy as np
import matplotlib.pyplot as plt
import maup
import gerrychain
from gerrychain.updaters import cut_edges, Tally
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.optimization import SingleMetricOptimizer, Gingleator
from gerrychain.tree import recursive_seed_part
from functools import partial
import pandas as pd
import json
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import random

AIANHH_data = pd.read_csv("national_block_assignment_aianhh/national_block_assignment_aianhh.csv")
national_2020_results_to_blocks = pd.read_csv("national_block_2020_pres_results/national_block_2020_pres_results.csv")




/tmp/ipykernel_4494/1222863903.py:22: DtypeWarning: Columns (9,10,12,15,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  AIANHH_data = pd.read_csv("national_block_assignment_aianhh/national_block_assignment_aianhh.csv")


## Import Shapefiles
Put a block level TIGER/Line shape file into `state_block_gdf` and a precicnt level shape file with results into the `state_precinct_gdf`. Precinct shapefiles often have to be repaired, so run it through `Repairs.ipynb` first. Also import P1 demographic data from the census at the block level for the target state.

In [2]:
state_block_gdf = gpd.read_file("NewMexico/tl_2020_35_tabblock20/tl_2020_35_tabblock20.shp")
state_precinct_gdf = gpd.read_file("NewMexico/NM_repaired/NM_repaired.shp")
state_racial_data = pd.read_csv("NewMexico/P1_NM_Data/DECENNIALPL2020.P1-Data.csv")

/tmp/ipykernel_4494/3382807585.py:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  state_racial_data = pd.read_csv("NewMexico/P1_NM_Data/DECENNIALPL2020.P1-Data.csv")


## Prepare for Merging

In [3]:
state_racial_data.drop(0, inplace=True)
state_racial_data['GEO_ID'] = state_racial_data['GEO_ID'].str[9:26]
state_racial_data

,GEO_ID,NAME,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,...,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73
1,350010001071000,"Block 1000, Block Group 1, Census Tract 1.07, ...",343,307,289,0,0,15,0,3,...,0,0,0,0,0,0,0,0,0,NaN
2,350010001071001,"Block 1001, Block Group 1, Census Tract 1.07, ...",17,17,10,1,0,0,0,6,...,0,0,0,0,0,0,0,0,0,NaN
3,350010001071002,"Block 1002, Block Group 1, Census Tract 1.07, ...",46,39,30,0,4,4,0,1,...,0,0,0,0,0,0,0,0,0,NaN
4,350010001071003,"Block 1003, Block Group 1, Census Tract 1.07, ...",36,33,32,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
5,350010001071004,"Block 1004, Block Group 1, Census Tract 1.07, ...",35,23,19,0,0,1,0,3,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107211,350619803001004,"Block 1004, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
107212,350619803001005,"Block 1005, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
107213,350619803001006,"Block 1006, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
107214,350619803001007,"Block 1007, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


Change numbers to ints for merging.

In [4]:
state_racial_data['GEOID20'] = state_racial_data['GEO_ID'].astype(int)
state_racial_data['P1_005N']=state_racial_data['P1_005N'].astype(int)
state_racial_data['P1_001N']=state_racial_data['P1_001N'].astype(int)
state_racial_data['P1_012N']=state_racial_data['P1_012N'].astype(int)
state_racial_data

,GEO_ID,NAME,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,...,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73,GEOID20
1,350010001071000,"Block 1000, Block Group 1, Census Tract 1.07, ...",343,307,289,0,0,15,0,3,...,0,0,0,0,0,0,0,0,NaN,350010001071000
2,350010001071001,"Block 1001, Block Group 1, Census Tract 1.07, ...",17,17,10,1,0,0,0,6,...,0,0,0,0,0,0,0,0,NaN,350010001071001
3,350010001071002,"Block 1002, Block Group 1, Census Tract 1.07, ...",46,39,30,0,4,4,0,1,...,0,0,0,0,0,0,0,0,NaN,350010001071002
4,350010001071003,"Block 1003, Block Group 1, Census Tract 1.07, ...",36,33,32,0,0,0,0,1,...,0,0,0,0,0,0,0,0,NaN,350010001071003
5,350010001071004,"Block 1004, Block Group 1, Census Tract 1.07, ...",35,23,19,0,0,1,0,3,...,0,0,0,0,0,0,0,0,NaN,350010001071004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107211,350619803001004,"Block 1004, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,350619803001004
107212,350619803001005,"Block 1005, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,350619803001005
107213,350619803001006,"Block 1006, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,350619803001006
107214,350619803001007,"Block 1007, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,350619803001007


In [5]:
state_block_gdf['GEOID20']=state_block_gdf['GEOID20'].astype(int)

### Begin Merges

In [6]:
state_block_gdf = state_block_gdf.merge(AIANHH_data, on = ['GEOID20'], how = 'left')

In [7]:
state_block_gdf = state_block_gdf.merge(state_racial_data, on = ['GEOID20'], how = 'left')
state_block_gdf

,STATEFP20_x,COUNTYFP20_x,TRACTCE20_x,BLOCKCE20_x,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73
0,35,035,000305,2014,350350003052014,Block 2014,G5040,U,00820,U,...,0,0,0,0,0,0,0,0,0,NaN
1,35,035,000904,4521,350350009044521,Block 4521,G5040,R,None,None,...,0,0,0,0,0,0,0,0,0,NaN
2,35,035,000602,1006,350350006021006,Block 1006,G5040,R,None,None,...,0,0,0,0,0,0,0,0,0,NaN
3,35,035,000904,4399,350350009044399,Block 4399,G5040,R,None,None,...,0,0,0,0,0,0,0,0,0,NaN
4,35,035,000701,2007,350350007012007,Block 2007,G5040,R,None,None,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107210,35,053,978100,1409,350539781001409,Block 1409,G5040,R,None,None,...,0,0,0,0,0,0,0,0,0,NaN
107211,35,045,000706,2143,350450007062143,Block 2143,G5040,R,None,None,...,0,0,0,0,0,0,0,0,0,NaN
107212,35,015,000900,1666,350150009001666,Block 1666,G5040,R,None,None,...,0,0,0,0,0,0,0,0,0,NaN
107213,35,053,978200,2066,350539782002066,Block 2066,G5040,R,None,None,...,0,0,0,0,0,0,0,0,0,NaN


In [ ]:
# Bespoke division function to prevent ZeroDivisionError
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 0
state_block_gdf['native_american_perc'] = divide(state_block_gdf['P1_005N'],state_block_gdf['P1_001N'])
state_block_gdf['mixed_native_american_perc'] = divide(state_block_gdf['P1_012N'],state_block_gdf['P1_001N'])
state_block_gdf['non_native_perc'] = (state_block_gdf['P1_005N']-(state_block_gdf['P1_005N']+state_block_gdf['P1_012N']))
state_block_gdf

,STATEFP20_x,COUNTYFP20_x,TRACTCE20_x,BLOCKCE20_x,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73,native_american_perc,mixed_native_american_perc,non_native_perc
0,35,035,000305,2014,350350003052014,Block 2014,G5040,U,00820,U,...,0,0,0,0,0,0,NaN,0.000000,0.028571,-1
1,35,035,000904,4521,350350009044521,Block 4521,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
2,35,035,000602,1006,350350006021006,Block 1006,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
3,35,035,000904,4399,350350009044399,Block 4399,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
4,35,035,000701,2007,350350007012007,Block 2007,G5040,R,None,None,...,0,0,0,0,0,0,NaN,0.052632,0.157895,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107210,35,053,978100,1409,350539781001409,Block 1409,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
107211,35,045,000706,2143,350450007062143,Block 2143,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
107212,35,015,000900,1666,350150009001666,Block 1666,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
107213,35,053,978200,2066,350539782002066,Block 2066,G5040,R,None,None,...,0,0,0,0,0,0,NaN,0.000000,0.272727,-3


### Assign Block data to Precincts

In [10]:
#Change the coordiante system for maup.assign()
state_precinct_gdf.crs
state_precinct_gdf = state_precinct_gdf.to_crs(4269)

In [11]:
sum_variables = ["P1_001N", "P1_005N", "P1_012N"]

#Assign our blocks to precincts
blocks_to_precincts_assignment = maup.assign(state_block_gdf.geometry, state_precinct_gdf.geometry)


/home/gabed/Documents/Fall_24/Math195/Final_Project/.conda/lib/python3.10/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/home/gabed/Documents/Fall_24/Math195/Final_Project/.conda/lib/python3.10/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/home/gabed/Documents/Fall_24/Math195/Final_Project/.conda/lib/python3.10/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return 

Check if every block got assigned to a column

In [12]:
state_block_gdf[blocks_to_precincts_assignment.isna()]

,STATEFP20_x,COUNTYFP20_x,TRACTCE20_x,BLOCKCE20_x,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73,native_american_perc,mixed_native_american_perc,non_native_perc


In [13]:
state_block_gdf

,STATEFP20_x,COUNTYFP20_x,TRACTCE20_x,BLOCKCE20_x,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73,native_american_perc,mixed_native_american_perc,non_native_perc
0,35,035,000305,2014,350350003052014,Block 2014,G5040,U,00820,U,...,0,0,0,0,0,0,NaN,0.000000,0.028571,-1
1,35,035,000904,4521,350350009044521,Block 4521,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
2,35,035,000602,1006,350350006021006,Block 1006,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
3,35,035,000904,4399,350350009044399,Block 4399,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
4,35,035,000701,2007,350350007012007,Block 2007,G5040,R,None,None,...,0,0,0,0,0,0,NaN,0.052632,0.157895,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107210,35,053,978100,1409,350539781001409,Block 1409,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
107211,35,045,000706,2143,350450007062143,Block 2143,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
107212,35,015,000900,1666,350150009001666,Block 1666,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,0
107213,35,053,978200,2066,350539782002066,Block 2066,G5040,R,None,None,...,0,0,0,0,0,0,NaN,0.000000,0.272727,-3


##### Grouping our demographic data together by precinct

In [14]:
state_precinct_gdf[sum_variables] = state_block_gdf[sum_variables].groupby(blocks_to_precincts_assignment).sum()
state_precinct_gdf

,STATEFP,COUNTYFP,COUNTYNAME,VTDST,NAME,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREGHAW,G20PRECTIT,...,G20SACRJOH,G20SACDHEN,G20SACRLEE,G20SACLCUR,G20SACDYOH,G20SACRMON,geometry,P1_001N,P1_005N,P1_012N
0,35,001,Bernalillo,1,PRECINCT 001,556,439,20,3,4,...,466,467,418,85,495,475,"POLYGON ((-106.75235 35.20291, -106.75248 35.2...",2090,126,33
1,35,001,Bernalillo,2,PRECINCT 002,406,290,17,3,0,...,319,345,300,49,368,324,"POLYGON ((-106.67201 35.21794, -106.67079 35.2...",1132,19,21
2,35,001,Bernalillo,3,PRECINCT 003,180,179,2,2,2,...,190,146,172,28,150,196,"POLYGON ((-106.66817 35.18102, -106.66815 35.1...",614,16,15
3,35,001,Bernalillo,35,PRECINCT 035,256,113,7,1,0,...,122,227,99,30,225,131,"POLYGON ((-106.70489 35.08974, -106.70420 35.0...",1164,55,14
4,35,001,Bernalillo,59,PRECINCT 059,329,167,4,3,0,...,181,283,150,39,284,184,"POLYGON ((-106.74273 35.07272, -106.74264 35.0...",2296,159,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,35,041,Roosevelt,4,PRECINCT 004,27,232,2,0,0,...,223,28,206,13,28,218,"POLYGON ((-103.51002 33.57012, -103.51013 33.5...",455,2,4
1913,35,041,Roosevelt,18,PRECINCT 018,69,371,2,0,0,...,360,63,334,21,65,353,"POLYGON ((-103.14357 34.30296, -103.14324 34.3...",1059,14,15
1914,35,005,Chaves,45,PRECINCT 045,2,16,0,0,0,...,16,2,16,0,2,16,"POLYGON ((-104.23934 33.62644, -104.23944 33.6...",47,1,0
1915,35,061,Valencia,54,PRECINCT 054,206,220,9,2,1,...,241,187,204,32,180,241,"POLYGON ((-106.75173 34.80468, -106.75188 34.8...",1136,47,14


### Taking a weighted average of candidate support percentages

In [15]:
election_columns = ["G20PRERTRU", "G20PREDBID"]

In [16]:
weights = state_block_gdf["P1_001N"]/blocks_to_precincts_assignment.map(state_block_gdf["P1_001N"].groupby(blocks_to_precincts_assignment).sum())
weights = weights.fillna(0)
weights

0         0.034347
1         0.000000
2         0.000000
3         0.000000
4         0.015200
            ...   
107210    0.000000
107211    0.000000
107212    0.000000
107213    0.013836
107214    0.008893
Length: 107215, dtype: float64

In [17]:
state_precinct_gdf.columns

Index(['STATEFP', 'COUNTYFP', 'COUNTYNAME', 'VTDST', 'NAME', 'G20PREDBID',
       'G20PRERTRU', 'G20PRELJOR', 'G20PREGHAW', 'G20PRECTIT', 'G20PRESLAR',
       'G20USSDLUJ', 'G20USSRRON', 'G20USSLWAL', 'G20SSCDBAC', 'G20SSCRFUL',
       'G20SSCDTHO', 'G20SSCRMOR', 'G20SACDIVE', 'G20SACRJOH', 'G20SACDHEN',
       'G20SACRLEE', 'G20SACLCUR', 'G20SACDYOH', 'G20SACRMON', 'geometry',
       'P1_001N', 'P1_005N', 'P1_012N'],
      dtype='object')

In [18]:
prorated=maup.prorate(blocks_to_precincts_assignment, state_precinct_gdf[election_columns], weights)
state_block_gdf[election_columns] = prorated

### Export 
Export our geodataframe as a shapefile for ensemble analysis in `Neutral_Ensembleizer.ipynb` or `Optimizer.ipynb`

In [19]:
state_precinct_gdf.to_file("NewMexico/NM_ready_4_ensemble")